# Data analysis of UDN patients

### Connect to the UDN data resource using the HPDS Adapter

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
import scipy.stats as st
import scipy.sparse as sp
import networkx as nx
from community import community_louvain
from scipy.stats import kruskal
import seaborn as sns
import collections as collec
import os
import xml.etree.ElementTree as ET

In [ ]:
# matplotlib.pyplot settings for font size of figures
font = {'size'   : 30}
plt.rc('font', **font)

In [ ]:
import PicSureHpdsLib
import PicSureClient
# Connection to the PicSure Client w/ token: private token for people with authorized access
connection = PicSureClient.Client.connect("https://udn.hms.harvard.edu/picsure", token)
adapter = PicSureHpdsLib.Adapter(connection)
resource = adapter.useResource("8e8c7ed0-87ea-4342-b8da-f939e46bac26")

In [ ]:
def removekey(d, key):
    """This functions returns a copy of a dictionnary with a removed key
    Parameters: d : dictionnary
                key: the key that must be deleted
    Returns: copy of dictionnary d without the key 
    """
    r = dict(d)
    del r[key]
    return r

In [ ]:
def get_CI(a):
    """Returns the 95% confidence interval for a list/array a
    Parameters: a: list or array we want the CI for
    Returns: a tuple with 95% confidence interval
    """
    return st.t.interval(0.95, len(a)-1, loc=np.mean(a), scale=st.sem(a,nan_policy='omit'))

### Download data

In [ ]:
def get_data_df(column_head):
    """Enables the user to download the data as a pandas dataframe indexed by UDN IDs (through API)
    Parameters : column_head : string, with the name of the header that will be selected. For example, if the columns that 
                                should be selected containt "this string", then column_head="this string".
    Returns: df : dataframe indexed by UDN IDs of the selected columns
    """
    dictionary=resource.dictionary().find(column_head)
    query=resource.query()
    query.select().add(dictionary.keys())
    query.select().add('\\000_UDN ID\\')
    df=query.getResultsDataFrame()
    df.set_index("\\000_UDN ID\\", inplace=True)
    query.select().clear()
    return df

### Download phenotypic, status, genomic, primary symptoms and meta- data

In [ ]:
phenotypes = get_data_df("\\04_Clinical symptoms and physical findings (in HPO, from PhenoTips)\\")

In [ ]:
# select only the phenotypes, and not the prenatal phenotypes
columns_to_del=[]
for col in list(phenotypes.columns)[1:]:
    if "Prenatal Phenotype" in col.split('\\'):
        columns_to_del.append(col)

In [ ]:
phenotypes=phenotypes.drop(columns_to_del,axis=1)

In [ ]:
status = get_data_df("\\13_Status\\")

In [ ]:
genes=get_data_df("\\11_Candidate genes\\")
variants=get_data_df("\\12_Candidate variants\\")

In [ ]:
primary_symptoms=get_data_df("\\01_Primary symptom category reported by patient or caregiver\\")

In [ ]:
clinical_site=get_data_df('\\03_UDN Clinical Site\\')

In [ ]:
family_history=get_data_df("\\08_Family history (from PhenoTips)\\")

In [ ]:
natal_history=get_data_df("\\09_Prenatal and perinatal history (from PhenoTips)\\")

In [ ]:
demographics=get_data_df("\\00_Demographics\\")

In [ ]:
diagnostics=get_data_df('\\14_Disorders (in OMIM, from PhenoTips)\\')

### HPO analysis

In [ ]:
def get_patient_phenotypes(phenotypes):
    """Gets the list of unique phenotypes presented by the patients of the UDN 
    Parameters: phenotypes : pandas dataframe with the phenotypes
    
    Returns : patient_phen : dictionary with patients as keys, with values being dictionaries with keys ("pos","neg") 
                             with a list of the positive and negative phenotypes presented by each patient
    """
    header_phen=list(phenotypes)
    patient_phen={patient: {"pos": [], "neg": []} for patient in phenotypes.index.values}
    for patient,row in phenotypes.iterrows():
        for i,phen in enumerate(row):
            if phen=="Positive":
                if not header_phen[i].split("\\")[-2] in patient_phen[patient]["pos"]:
                    patient_phen[patient]["pos"].append(header_phen[i].split("\\")[-2])
            elif phen=="Negative":
                if not header_phen[i].split("\\")[-2] in patient_phen[patient]["neg"]:
                    patient_phen[patient]["neg"].append(header_phen[i].split("\\")[-2])

    for patient in patient_phen:
        if len(patient_phen[patient]["pos"])==0 and len(patient_phen[patient]["neg"])==0:
            patient_phen=removekey(patient_phen,patient)
    return patient_phen

In [ ]:
patient_phen=get_patient_phenotypes(phenotypes)
patient_phen

In [ ]:
# get the counts of positivie, negative and total HPO terms
HPO_terms_pos,HPO_terms_neg={patient: len(patient_phen[patient]["pos"]) for patient in patient_phen},{patient: len(patient_phen[patient]["neg"]) for patient in patient_phen}
HPO_terms={patient: len(patient_phen[patient]["pos"])+len(patient_phen[patient]["neg"]) for patient in patient_phen}

In [ ]:
# get the list of diagnosed and undiagnosed patients
list_diagnosed=status.loc[status["\\13_Status\\"] == "solved"].index.values.tolist()
list_undiagnosed=status.loc[status["\\13_Status\\"] != "solved"].index.values.tolist()

In [ ]:
# get the list of diagnosed or undiagnosed patients that have at least one HPO term
list_diagnosed_phen=[patient for patient in list_diagnosed if(patient in patient_phen)]
list_undiagnosed_phen=[patient for patient in list_undiagnosed if(patient in patient_phen)]

In [ ]:
# get the dataframes with the phenotypes of diagnosed or undiagnosed patients that have at least one HPO term
phenotypes_diagnosed=phenotypes.loc[list_diagnosed_phen]
phenotypes_undiagnosed=phenotypes.loc[list_undiagnosed_phen]

In [ ]:
# transform the HPO counts into a list of values 
HPO_list_pos=[HPO_terms_pos[patient] for patient in HPO_terms_pos]
HPO_list_neg=[HPO_terms_neg[patient] for patient in HPO_terms_neg]
HPO_list=[HPO_terms[patient] for patient in HPO_terms]

In [ ]:
# show the total number of positive, negative, and all HPO terms in the database
print("# of positive HPO : ",np.sum(HPO_list_pos),"# of negative HPO : ",np.sum(HPO_list_neg),"# of total HPO : ",np.sum(HPO_list))

In [ ]:
# get the list of HPO terms # for diagnosed patients
HPO_list_pos_d=[HPO_terms_pos[patient] for patient in list_diagnosed_phen]
HPO_list_neg_d=[HPO_terms_neg[patient] for patient in list_diagnosed_phen]
HPO_list_d=[HPO_terms[patient] for patient in list_diagnosed_phen]

In [ ]:
# get the list of HPO terms # for undiagnosed patients
HPO_list_pos_nd=[HPO_terms_pos[patient] for patient in list_undiagnosed_phen]
HPO_list_neg_nd=[HPO_terms_neg[patient] for patient in list_undiagnosed_phen]
HPO_list_nd=[HPO_terms[patient] for patient in list_undiagnosed_phen]

In [ ]:
def show_stats_HPO_counts(HPO_list,HPO_list_pos,HPO_list_neg):
    """Show the average and confidence interval for HPO terms for a selected population
    Parameters: HPO_list: list of HPO # for selected population
                HPO_list_pos: list of positive HPO # for selected population
                HPO_list_neg: list of negative HPO # for selected population
    Returns: None
    Shows the average and CI 95% for HPO counts
    """
    print("HPO pos average : ",np.average(HPO_list_pos),", CI 95% : ",get_CI(HPO_list_pos),", HPO pos max : ",np.max(HPO_list_pos))
    print("HPO neg average : ",np.average(HPO_list_neg),", CI 95% : ",get_CI(HPO_list_neg),", HPO neg max : ",np.max(HPO_list_neg))
    print("HPO average : ",np.average(HPO_list),", CI 95% : ",get_CI(HPO_list),", HPO max : ",np.max(HPO_list))

In [ ]:
show_stats_HPO_counts(HPO_list,HPO_list_pos,HPO_list_neg)

In [ ]:
show_stats_HPO_counts(HPO_list_d,HPO_list_pos_d,HPO_list_neg_d)

In [ ]:
show_stats_HPO_counts(HPO_list_nd,HPO_list_pos_nd,HPO_list_neg_nd)

In [ ]:
def show_distrib_HPO(HPO_list,name):
    """Plots the distribution of count of HPO terms per patient
    Parameters : HPO_list: list of counts for each patient of HPO terms
                 name: string, title of the figure
    Returns : None
    Shows matplotlib plot of distribution of HPO
    """
    distrib=collec.Counter(HPO_list)
    X=[key for key in distrib.keys()]
    Y=[distrib[key] for key in distrib.keys()]
    plt.figure(figsize=(20,15))
    plt.plot(X,Y,"o")
    plt.xlabel("Number of HPO terms")
    plt.ylabel("Count of patients")
    plt.title(name)
    plt.yscale("log")
    plt.xscale("log")
    plt.axes().set_ylim(None,200)
    plt.show()
    plt.savefig("HPO_terms_log")

In [ ]:
show_distrib_HPO(HPO_list,"Distribution of HPO terms")
show_distrib_HPO(HPO_list_neg,"Distribution of negative HPO terms")
show_distrib_HPO(HPO_list_pos,"Distribution of positive HPO terms")

### HPO large group stats

In [ ]:
# get the list of large groups in the HPO hierarchy
large_groups_HPO=[]
header_phen=list(phenotypes)[1:]
for phen in header_phen:
    if not(phen.split("\\")[4] in large_groups_HPO):
        large_groups_HPO.append(phen.split("\\")[4])
large_groups_HPO  

In [ ]:
def get_large_groups_HPO_count(large_groups_HPO,phenotypes):
    """Returns the count of HPO terms that belong to a certain group of HPO terms
    Parameters: large_groups : list of large groups that belong to the HPO hierarchy
                phenotypes : pandas dataframe with the phenotypes
    
    Returns : group_count : dictionary with keys ("pos","neg") that counts the occurrences of positive or negative HPO terms
                            for each large group
    """
    header_phen=list(phenotypes)
    group_count={"pos":{lg: 0 for lg in large_groups_HPO},"neg": {lg: 0 for lg in large_groups_HPO}}
    for patient,row in phenotypes.iterrows():
        for i,phen in enumerate(row):
            if phen=="Positive":
                group_count["pos"][header_phen[i].split("\\")[4]]+=1
            elif phen=="Negative":
                group_count["neg"][header_phen[i].split("\\")[4]]+=1
    return group_count

In [ ]:
large_groups_HPO_count=get_large_groups_HPO_count(large_groups_HPO,phenotypes)
large_groups_HPO_count

In [ ]:
# get the count of large groups for positive and negative terms of diagnosed patients
large_groups_HPO_count_diagnosed=get_large_groups_HPO_count(large_groups_HPO,phenotypes_diagnosed)
large_groups_HPO_count_diagnosed

In [ ]:
# get the count of large groups for positive and negative terms of undiagnosed patients
large_groups_HPO_count_undiagnosed=get_large_groups_HPO_count(large_groups_HPO,phenotypes_undiagnosed)
large_groups_HPO_count_undiagnosed

### Comparison HPO and Primary Symptoms


In [ ]:
# get the association between unique phenotypes and the large groups they are related to in the HPO hierarchy
# list_phenotypes_unique is a dictionary with the phenotypes as keys, and a list of associated large groups as value
list_phenotypes_unique={}
for phen in header_phen:
    if not(phen.split("\\")[-2] in list_phenotypes_unique):
        list_phenotypes_unique[phen.split("\\")[-2]]=[phen.split("\\")[4]]
    else:
        if not(phen.split("\\")[4] in list_phenotypes_unique[phen.split("\\")[-2]]):
            list_phenotypes_unique[phen.split("\\")[-2]].append(phen.split("\\")[4])
list_phenotypes_unique

In [ ]:
def get_link_between_PS_HPO(patient_phen,primary_symptoms,list_phenotypes_unique):
    """Returns the link count of occurrence of a certain HPO large group for patients with a certain primary symptom
    Parameters : patient_phen: dictionary with the positive or negative unique HPO terms linked to patients 
                 primary_symptoms: dataframe with UDN IDs as index, and list of primary symptoms reported 
                 list_phenotypes_unique: dictionary of link between phenotypes and the large groups they are linked
                 to in the HPO hierarchy
    Returns : dictionary with keys ("pos","neg") that contain a dictionary with the primary symptoms as keys and a dictionary 
              with the count for every large group of HPO hierarchy of occurrences as value
    """
    link_PS_HPO={"pos": {}, "neg": {}}
    for patient in patient_phen:
        ps=list(primary_symptoms.loc[patient])[1]
        if not(ps in link_PS_HPO["pos"]):
            link_PS_HPO["pos"][ps]={}
        if not(ps in link_PS_HPO["neg"]):
            link_PS_HPO["neg"][ps]={}
        for phen in patient_phen[patient]["pos"]:
            for lg in list_phenotypes_unique[phen]:
                if lg in link_PS_HPO["pos"][ps]:
                    link_PS_HPO["pos"][ps][lg]+=1
                else:
                    link_PS_HPO["pos"][ps][lg]=1
        for phen in patient_phen[patient]["neg"]:
            for lg in list_phenotypes_unique[phen]:
                if lg in link_PS_HPO["neg"][ps]:
                    link_PS_HPO["neg"][ps][lg]+=1
                else:
                    link_PS_HPO["neg"][ps][lg]=1
    return link_PS_HPO

In [ ]:
# get the links between the primary symptoms and the HPO large groups
link_PS_HPO=get_link_between_PS_HPO(patient_phen,primary_symptoms,list_phenotypes_unique)
link_PS_HPO

In [ ]:
# Show the ranked HPO groups for each primary symptom 
for ps in link_PS_HPO["pos"]:
    print("Primary symptom ",ps)
    print("-------------------------------------------")
    if type(ps)==float:
        continue
    lg_list=list(link_PS_HPO["pos"][ps])
    val=[link_PS_HPO["pos"][ps][lg] for lg in lg_list]
    indsort=np.argsort(val)[::-1]
    lg_list=np.array(lg_list)[indsort]
    val=np.array(val)[indsort]
    for i in range(len(indsort)):
        print(lg_list[i],val[i])
    print("-------------------------------------------")

### Analysis of demographics and clinical site

In [ ]:
# get the dataframes for patients with at least one phenotype, for diagnosed and undiagnosed 
demographics = demographics.loc[list(patient_phen)]
demographics_diagnosed = demographics.loc[list_diagnosed_phen]
demographics_undiagnosed = demographics.loc[list_undiagnosed_phen]
clinical_site = clinical_site.loc[list(patient_phen)]
clinical_site_diagnosed = clinical_site.loc[list_diagnosed_phen]
clinical_site_undiagnosed = clinical_site.loc[list_undiagnosed_phen]

In [ ]:
# get count of clinical sites for patients with at least one phenotype, for diagnosed and undiagnosed
cscount = clinical_site.groupby('\\03_UDN Clinical Site\\')['Patient ID'].nunique()
cscount_d = clinical_site_diagnosed.groupby('\\03_UDN Clinical Site\\')['Patient ID'].nunique()
cscount_nd = clinical_site_undiagnosed.groupby('\\03_UDN Clinical Site\\')['Patient ID'].nunique()

In [ ]:
print("Clinical site count general")
print(cscount)
print("Clinical site count diagnosed")
print(cscount_d)
print("Clinical site count undiagnosed")
print(cscount_nd)

In [ ]:
print("Count race for general ",collec.Counter(demographics['\\00_Demographics\\Ethnicity\\']))
print("Count race for diagnosed ",collec.Counter(demographics_diagnosed['\\00_Demographics\\Ethnicity\\']))
print("Count race for undiagnosed ",collec.Counter(demographics_undiagnosed['\\00_Demographics\\Ethnicity\\']))

In [ ]:
print("Count race for general ",collec.Counter(demographics["\\00_Demographics\\Race\\"]))
print("Count race for diagnosed ",collec.Counter(demographics_diagnosed["\\00_Demographics\\Race\\"]))
print("Count race for undiagnosed ",collec.Counter(demographics_undiagnosed["\\00_Demographics\\Race\\"]))

In [ ]:
# get the statistics for demographics
demographics.describe()

In [ ]:
# get the statistics for demographics, for diagnosed patients
demographics_diagnosed.describe()

In [ ]:
# get the statistics for demographics, for undiagnosed patients
demographics_undiagnosed.describe()

In [ ]:
# get the gender count, for diagnosed and undiagnosed
gender_count = demographics.groupby("\\00_Demographics\\Gender\\")['Patient ID'].nunique()
gender_count_d = demographics_diagnosed.groupby("\\00_Demographics\\Gender\\")['Patient ID'].nunique()
gender_count_nd = demographics_undiagnosed.groupby("\\00_Demographics\\Gender\\")['Patient ID'].nunique()

In [ ]:
print("Gender count general")
print(gender_count)
print("Gender count diagnosed")
print(gender_count_d)
print("Gender count undiagnosed")
print(gender_count_nd)

### Count of primary symptoms 

In [ ]:
# get the primary symptoms for patients with at least one phenotype, for diagnosed and undiagnosed
primary_symptoms = primary_symptoms.loc[list(patient_phen)]
primary_symptoms_d = primary_symptoms.loc[list_diagnosed_phen]
primary_symptoms_nd = primary_symptoms.loc[list_undiagnosed_phen]

In [ ]:
# get the primary symptom count, for diagnosed and undiagnosed
pscount = primary_symptoms.groupby("\\01_Primary symptom category reported by patient or caregiver\\")['Patient ID'].nunique()
pscount_d = primary_symptoms_d.groupby("\\01_Primary symptom category reported by patient or caregiver\\")['Patient ID'].nunique()
pscount_nd = primary_symptoms_nd.groupby("\\01_Primary symptom category reported by patient or caregiver\\")['Patient ID'].nunique()

In [ ]:
print("Primary symptom count general")
print(pscount)
print("Primary symptom count diagnosed")
print(pscount_d)
print("Primary symptom count undiagnosed")
print(pscount_nd)

### Family history

In [ ]:
# get family history for patients with at least one phenotype, diagnosed or undiagnosed
family_history = family_history.loc[list(patient_phen)]
family_history_d = family_history.loc[list_diagnosed_phen]
family_history_nd = family_history.loc[list_undiagnosed_phen]

In [ ]:
# get count of affected relatives, for diagnosed or undiagnosed
fhcount = family_history.groupby("\\08_Family history (from PhenoTips)\\Affected Relatives\\")['Patient ID'].nunique()
fhcount_d = family_history_d.groupby("\\08_Family history (from PhenoTips)\\Affected Relatives\\")['Patient ID'].nunique()
fhcount_nd = family_history_nd.groupby("\\08_Family history (from PhenoTips)\\Affected Relatives\\")['Patient ID'].nunique()

In [ ]:
print("Affected relatives count general")
print(fhcount)
print("Affected relatives count diagnosed")
print(fhcount_d)
print("Affected relatives count undiagnosed")
print(fhcount_nd)

In [ ]:
# get natal history for patients with at least one phenotype
natal_history = natal_history.loc[list(patient_phen)]

In [ ]:
# replace missing values by NaN
natal_history = natal_history.replace(0, np.NaN)

In [ ]:
natal_history.count()

In [ ]:
natal_history.describe()

In [ ]:
# get the number of positive or negative occurrences for any given phenotype. Ex: if count_pos_phen[i]=3, 
# then there are three patients in the database that are positive for the phenotype header_phen[i]
count_pos_phen,count_neg_phen=[0 for i in range(1,phenotypes.shape[1])],[0 for i in range(1,phenotypes.shape[1])]
for i in range(1,phenotypes.shape[1]):
    cts=phenotypes.iloc[:,i].value_counts()
    keys=cts.keys().tolist()
    for j in range(len(keys)):
        if keys[j]=="Positive":
            count_pos_phen[i-1]=cts[j]
        elif keys[j]=="Negative":
            count_neg_phen[i-1]=cts[j]
    

In [ ]:
collec.Counter(family_history["\\08_Family history (from PhenoTips)\\Consanguinity\\"])

In [ ]:
def get_best_phenotypes_consang(patient_phen,family_history):
    """Gives the list of overrepresented phenotypes in the consanguineous community
    Parameters : patient_phen: dictionary of list of positive and negative phenotypes for each patient
                 family_history : dataframe with family history 
    Returns : dictionary with the count for positive or negative phenotypes of patients presenting such phenotype
    Shows the ranked 10 best phenotypes, for positive and negative as well as the Mann Whitney U stats for difference in 
    distribution between the consanguineous and general community
    """
    count_phenotype_consang={"pos": {}, "neg": {}}
    csgcount=0
    for patient in list(patient_phen):
        consang = family_history.loc[patient][2]
        if consang==True:
            csgcount+=1
            for phen_pos in patient_phen[patient]["pos"]:
                if not(phen_pos in count_phenotype_consang["pos"]):
                    count_phenotype_consang["pos"][phen_pos]=1
                else:
                    count_phenotype_consang["pos"][phen_pos]+=1
            for phen_neg in patient_phen[patient]["neg"]:
                if not(phen_neg in count_phenotype_consang["neg"]):
                    count_phenotype_consang["neg"][phen_neg]=1
                else:
                    count_phenotype_consang["neg"][phen_neg]+=1
    
    phen_pos_list=list(count_phenotype_consang["pos"])
    val=[count_phenotype_consang["pos"][phen] for phen in phen_pos_list]
    indsort=np.argsort(val)[::-1]
    phen_pos_list=np.array(phen_pos_list)[indsort][:18]
    val=np.array(val)[indsort][:18]
    print('Best positive phenotypes')
    comp_mw_true=[]
    for j,phen in enumerate(phen_pos_list):
        for i,p in enumerate(list(phenotypes)[1:]):
            if p.split("\\")[-2]==phen:
                print(phen,"consang % ",val[j]/csgcount*100," general % ",count_pos_phen[i]/phenotypes.shape[0]*100)
                comp_mw_true.append(count_pos_phen[i]/phenotypes.shape[0]*100)
                break
    print("Mann-Whitney pos : ")
    print("Medians : ",np.median(np.multiply(val,100/csgcount)),np.median(comp_mw_true))
    print(mannwhitneyu(np.multiply(val,100/csgcount),comp_mw_true))
    phen_neg_list=list(count_phenotype_consang["neg"])
    val=[count_phenotype_consang["neg"][phen] for phen in phen_neg_list]
    indsort=np.argsort(val)[::-1]
    phen_neg_list=np.array(phen_neg_list)[indsort][:10]
    val=np.array(val)[indsort][:10]
    print('Best negative phenotypes')
    comp_mw_true=[]
    for j,phen in enumerate(phen_neg_list):
        for i,p in enumerate(list(phenotypes)[1:]):
            if p.split("\\")[-2]==phen:
                print(phen,"consang % ",val[j]/csgcount*100," general % ",count_neg_phen[i]/phenotypes.shape[0]*100)
                comp_mw_true.append(count_neg_phen[i]/phenotypes.shape[0]*100)
                break
    print("Mann-Whitney neg : ")
    print("Medians : ",np.median(np.multiply(val,100/csgcount)),np.median(comp_mw_true))
    print(mannwhitneyu(np.multiply(val,100/csgcount),comp_mw_true))
    print("How many consang ?",csgcount)
    return count_phenotype_consang

In [ ]:
count_phenotype_consang=get_best_phenotypes_consang(patient_phen,family_history)

In [ ]:
# mat_age is the maternal age without the NaN values
mat_age=np.array(natal_history["\\09_Prenatal and perinatal history (from PhenoTips)\\Maternal Age\\"])
isnan_mat=np.isnan(mat_age)
mat_age=mat_age[[not(isnan_mat[i]) for i in range(len(isnan_mat))]]

In [ ]:
# distribution of maternal age in the US in 2009 (cf. article)
USA_dist = [3.1,6.9,24.4,28.2,23.1,11.5,2.8]
tranches=["0-18","18-19","20-24","24-29","30-34","34-39",">39"]

def distrib_age(mat_age, known_dist,tranches):
    """Shows the distribution of maternal age compared between UDN and the US in 2009
    Parameters : mat_age: array of maternal age in the UDN database
                 known_dist: list, known distribution of maternal age for age groups given in tranches
                 tranches: list of str, age groups that correspond to the known distribution 
    Returns: dictionary with age distribution in the UDN 
    Shows a joint plot of UDN distribution and known distribution of maternal age
    """
    count_age={}
    for age in mat_age:
        if age in count_age:
            count_age[age]+=1
        else:
            count_age[age]=1
    distrib_age=[0 for i in range(7)]
    for age in count_age:
        zone=-1
        if age<18:
            zone=0
        elif age>=18 and age <=19:
            zone=1
        elif age>19 and age<=24:
            zone=2
        elif age>24 and age<=29:
            zone=3
        elif age>29 and age<=34:
            zone=4
        elif age>34 and age<=39:
            zone=5
        else:
            zone=6
        distrib_age[zone]+=count_age[age]/len(mat_age)*100
    plt.figure(figsize=(20,20))
    plt.rcParams['font.size'] = 40
    plt.plot(tranches,distrib_age,'b',label="Distribution in UDN")
    plt.plot(tranches,known_dist,'r',label="Distribution in USA in 2009")
    plt.xlabel("Maternal age at birth")
    plt.ylabel("Distribution in UDN vs USA in 2009 (%)")
    plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
           ncol=2, mode="expand", borderaxespad=0.)
    plt.show()
    return distrib_age

dist_age_mat=distrib_age(mat_age,USA_dist,tranches)

ttest_ind(dist_age_mat,USA_dist)

### Genomics

In [ ]:
def get_gene_data(filename,var_or_gene):
    """Retrieve genetic data from a text file (formatted from JSON file)
    Parameters: filename: string, name of the text file with the genetic information
                var_or_gene: string, "Var" if variants of "Gen" if genes
    Returns: genomic_data: dictionary with UDN ID as key and list of dictionaries as value, each dictionary containing 
                           information about genes or variants
    """
    genomic_data={}
    with open(filename,"r") as pg:
        lines=pg.readlines()
        for line in lines:
            if line.split("<")[0]=="ID":
                pid=line.split(" ")[3].split("\n")[0]
                genomic_data[pid]=[]
            elif line.split("<")[0]==var_or_gene:
                var=int(line.split(" ")[1].split("\n")[0])
                genomic_data[pid].append({})
            else:
                if not(len(line.split(" "))==1):
                    genomic_data[pid][var][line.split(" ")[0].split("\n")[0]]=line.split(" ")[1].split("\n")[0]
    print(len(genomic_data))
    for patient in genomic_data:
        if not(patient in list(patient_phen.keys())):
            genomic_data=removekey(genomic_data,patient)
    print(len(genomic_data))
    return genomic_data

In [ ]:
variants=get_gene_data("patient_genomic.txt","Var")

In [ ]:
genes=get_gene_data("patient_genes.txt","Gene")

In [ ]:
# get the list of patients that present a candidate gene or candidate variants
list_patient_genes=list(genes.keys())
list_patient_variants=list(variants.keys())

In [ ]:
print("Patients in both", len([patient for patient in patient_phen if patient in list_patient_genes and patient in list_patient_variants]))
print("Patients with only genes", len([patient for patient in patient_phen if patient in list_patient_genes and not(patient in list_patient_variants)]))
print("Patients with only variants", len([patient for patient in patient_phen if not(patient in list_patient_genes) and patient in list_patient_variants]))

In [ ]:
# count the number of solved cases for people with an indicated gene or an indicated variant
print("Number of solved and unsolved cases for genes indicated : ",collec.Counter(status.loc[list(genes.keys())]["\\13_Status\\"]))
print("Number of solved and unsolved cases for variants indicated : ",collec.Counter(status.loc[list(variants.keys())]["\\13_Status\\"]))

In [ ]:
def get_dist_genomic(genomic_data,var_or_gene):
    """Get the distribution associated to genomic data for its characteristics
    Parameters: genomic_data: dictionary, with UDN ID as key and list with dictionaries as value, dict contaning characteristics
                              of the considered genomic data
                var_or_gene: string, "Var" if variants, "Gen" otherwise
    Returns: gene_effects: counter, with distribution of characteristics for selected genomic data
    """
    gene_list=[]
    for patient in genomic_data:
        for i in range(len(genomic_data[patient])):
            if var_or_gene=="Var":
                if "effect" in list(genomic_data[patient][i].keys()) and "gene" in list(genomic_data[patient][i].keys()):
                    gene_list.append([genomic_data[patient][i]["gene"],genomic_data[patient][i]["effect"]])
                else:
                    gene_list.append([genomic_data[patient][i]["gene"],"NA"])
            elif var_or_gene=="Gen":
                if "status" in list(genomic_data[patient][i].keys()) and "gene" in list(genomic_data[patient][i].keys()):
                    gene_list.append([genomic_data[patient][i]["gene"],genomic_data[patient][i]["status"]])
                else:
                    gene_list.append([genomic_data[patient][i]["gene"],"NA"])  
            else:
                print("var_or_gene must be Var or Gen")
    gene_effects=collec.Counter(np.array(gene_list)[:,1])
    return gene_effects

In [ ]:
# get the count of mutation types for candidate variants
gene_effects=get_dist_genomic(variants,"Var")
gene_effects

In [ ]:
# get the distribution of gene status for candidate genes
gene_status=get_dist_genomic(genes,"Gen")
gene_status

In [ ]:
def plot_distribution_genomic_data(genomic_data,namefile,var_or_gene):
    """Show the distribution of counts of candidate genes or variant per patient in the UDN database
    Parameters: genomic_data: dictionary, with UDN ID as key and list with dictionaries as value, dict contaning characteristics
                              of the considered genomic data
                namefile: string, file of the name to save the figure in 
                var_or_gene: string, "variants" if variants is considered, "genes" else
    Returns: None
    Show the distribution in a scatter plot and the counter, as well as total number of candidate genes/variants
    """
    count_gene_per_patient=collec.Counter([len(genomic_data[patient]) for patient in genomic_data])
    print(count_gene_per_patient)
    X_gene=list(count_gene_per_patient)
    Y_gene=[count_gene_per_patient[ct] for ct in X_gene]
    print("Number of total candidate ",var_or_gene," : ",np.sum([X_gene[i]*Y_gene[i] for i in range(len(X_gene))]))
    plt.plot(X_gene,Y_gene,"o")
    plt.title("Distribution of number of candidate "+var_or_gene+" per patient")
    plt.xlabel("Number of candidate "+var_or_gene)
    plt.ylabel("Count of patients")
    plt.savefig(namefile,bbox_inches="tight",dpi=300)
    plt.show()

In [ ]:
plot_distribution_genomic_data(variants,"Count_dist_var_per_pat.png","variants")

In [ ]:
plot_distribution_genomic_data(genes,"Count_genes_per_pat.png","genes")

In [ ]:
# This code to search for inconsistent genes vs variants: only for XXX does the variants and genes not match 
"""for patient in genes:
    if patient in variants:
        list_gen=[genes[patient][i]["id"] for i in range(len(genes[patient]))]
        list_var=[variants[patient][i]["gene"] for i in range(len(variants[patient]))]
        sub=(list_gen if len(list_gen)<=len(list_var) else list_var)
        supers=(list_var if len(list_gen)>=len(list_var) else list_gen)
        if not(set(sub).issubset(supers)):
            print(patient," genes ",list_gen," variants ",list_var)"""

### Statistics

In [ ]:
from scipy.stats import mannwhitneyu

All results are shown using the Mann Whitney U statistic. The closer to 0 the statistic, the more significantly different the distributions are -- this can also be assessed with the p value, if p<0.05 the distributions are significantly different

#### Diagnosed vs undiagnosed

In [ ]:
print("Age at UDN Evaluation")
mannwhitneyu(np.array(demographics_diagnosed["\\00_Demographics\\Age at UDN Evaluation (in years)\\"]),np.array(demographics_undiagnosed["\\00_Demographics\\Age at UDN Evaluation (in years)\\"]))

In [ ]:
print("Age at symptom onset")
mannwhitneyu(np.array(demographics_diagnosed["\\00_Demographics\\Age at symptom onset in years\\"]),np.array(demographics_undiagnosed["\\00_Demographics\\Age at symptom onset in years\\"]))


In [ ]:
# get corresponding sample sizes (although the Mann Whitney U accepts different sample sizes, 
# this gives greater power, since we have the information)
array_ps_d=np.array(pscount_d)
array_ps_d=np.insert(array_ps_d,8,0)
array_ps_d=np.insert(array_ps_d,12,0)
array_ps_d=np.insert(array_ps_d,18,0)
np.median(np.multiply(array_ps_d,1/len(list_diagnosed_phen))),np.median(np.multiply(np.array(pscount_nd),1/len(list_undiagnosed_phen)))

In [ ]:
print("Primary symptoms")
mannwhitneyu(np.multiply(array_ps_d,1/len(list_diagnosed_phen)),np.multiply(np.array(pscount_nd),1/len(list_undiagnosed_phen)))

In [ ]:
array_cs_d=np.insert(np.array(cscount_d),7,0)
np.median(np.multiply(array_cs_d,1/len(list_diagnosed_phen))),np.median(np.multiply(np.array(cscount_nd),1/len(list_undiagnosed_phen)))

In [ ]:
print("Clinical sites")
mannwhitneyu(np.multiply(array_cs_d,1/len(list_diagnosed_phen)),np.multiply(np.array(cscount_nd),1/len(list_undiagnosed_phen)))

# Clustering


In [ ]:
# get the index of unique phenotypes in the phenotype Dataframe
mat_phen_ind=[]
uniquep=[]
for i,phen in enumerate(header_phen):
    if not(phen.split("\\")[-2] in uniquep):
        mat_phen_ind.append(i)
        uniquep.append(phen.split("\\")[-2])
len(mat_phen_ind)

In [ ]:
# take the patient ID column out of the phenotype dataframe
matrix_phen=phenotypes.drop("Patient ID",axis=1)

In [ ]:
# transform the phenotype dataframe to obtain a matrix of unique phenotypes, with only patients that have been evaluated,
# with 1 if the phenotype is positively present, 0 if negative or NaN
mat_phen=matrix_phen.iloc[:,mat_phen_ind]
mat_phen=mat_phen.loc[list(patient_phen.keys())]
mat_phen=mat_phen.replace(to_replace={"Positive": 1, "Negative": 0, np.nan: 0})

In [ ]:
# The matrix is comprised of 1042 patients with at least 1 phenotype, and 3965 unique phenotypes
mat_phen.shape

In [ ]:
# we compute the jaccard similarity matrix for the phenotypic matrix
from sklearn.metrics.pairwise import pairwise_distances
jac_sim_un = 1 - pairwise_distances(mat_phen, metric = "jaccard")

In [ ]:
def graph_of_patients_js(UDN_IDs,sim_matrix):
    """Constructs the graph of UDN patients using the similarity matrix computed: nodes are patients, edges between patient
    i and j is proportional to the similarity between these two patients
    Parameters: UDN_IDs: list of UDN IDs of patients to consider
                sim_matrix: array, similarity matrix of pairwise similarity between each patient
    Returns : G: networkx graph of UDN patients 
              pos: array, positions of nodes 
    """
    G= nx.Graph()
    elist=[]
    print("udnlen",len(UDN_IDs))
    print("cslen",len(sim_matrix))
    for i in range(sim_matrix.shape[0]):
        G.add_node(UDN_IDs[i])
        for j in range(i,sim_matrix.shape[1]):
            elist.append((UDN_IDs[i],UDN_IDs[j],sim_matrix[i,j]))
    G.add_weighted_edges_from(elist)
    pos=nx.spring_layout(G,dim=2)
    return G,pos

In [ ]:
graph_un,pos_un=graph_of_patients_js(list(patient_phen.keys()),jac_sim_un)

In [ ]:
# writes the computed graph in a gml format, to be able to use Gephi to analyze it further
nx.write_gml(graph_un,"graph_un.gml")

In [ ]:
def compute_clusters_community(graph):
    """Compute the clusters in a graph using Louvain's community detection method
    Parameters : graph: networkx graph of UDN patients computed using the pairwise similarity between patients
    Returns: clusters: dictionary with the cluster number as key and a list containing all the patients in the cluster
                       as value
    """
    partition = community_louvain.best_partition(graph)
    print("Partition done")
    clusters={}
    for node in partition.keys():
        if not(partition[node] in clusters.keys()):
            clusters[partition[node]]=[node]
        else:
            clusters[partition[node]].append(node)
    count=0
    for cluster in clusters.keys():
        print("Length of cluster ",cluster,":",len(clusters[cluster]))
        if len(clusters[cluster])==1:
            count+=1
    print("Number of clusters with only one patient (outliers) :",count)
    return clusters

In [ ]:
clusters_un=compute_clusters_community(graph_un)

In [ ]:
#save the cluster compositions in a .txt format
with open("clusters_un.txt","w") as clus:
    for cluster in clusters_un:
        clus.write("<!> Cluster "+str(cluster)+"\n")
        for patient in clusters_un[cluster]:
            clus.write(patient+"\n")
clus.close()

In [ ]:
# we compute the indices of clusters with more than 2 patients, the indices of pairs and the indices of groups with only 1
ind_groups=[cluster for cluster in clusters_un if len(clusters_un[cluster])>2]
ind_pairs=[cluster for cluster in clusters_un if len(clusters_un[cluster])==2]
ind_outliers=[cluster for cluster in clusters_un if len(clusters_un[cluster])==1]
ind_groups,ind_pairs,ind_outliers

### Pair analysis

In [ ]:
# Check if there are phenotypes in common between pairs; if not, prints "Nothing in common"; if so, prints the common phenotype
for cluster in ind_pairs:
    print("Cluster C",cluster)
    cmn=False
    for phen in patient_phen[clusters_un[cluster][0]]["pos"]:
        if phen in patient_phen[clusters_un[cluster][1]]["pos"]:
            print("This phenotype in both patients: ",phen)
            cmn=True
    if not(cmn):
        print("Nothing in common")

In [ ]:
# Shows the phenotypes for the two pairs that have phenotypes in common
for cluster in [7,11]:
    print("Cluster C",cluster)
    print(patient_phen[clusters_un[cluster][0]],patient_phen[clusters_un[cluster][1]])

### Cluster analysis

In [ ]:
def calculate_diag_OR(clusters,clusters_ind,status):
    """Calculate the Odds Ratio for the probability of being diagnosed linked to being in a certain cluster
    Parameters: clusters: dictionary with cluster number as key and list of patients in cluster as value
                clusters_ind: list, indices of cluster to take into account 
                status: string, status of the patient (if patient's case is solved or not)
    Returns: OR_diag: dictionary with cluster number as key and the Odds Ratio (OR) for each cluster
    """
    count_diag_clusters={cluster: 0 for cluster in clusters_ind}
    for cluster in clusters_ind:
        for patient in clusters[cluster]:
            if status.loc[patient]["\\13_Status\\"]=="solved":
                count_diag_clusters[cluster]+=1
    OR_diag,IC={},{}
    def IC_func(sign,OR,a,b,c,d):
        if sign=="up":
            return np.exp(np.log(OR)+1.96*np.sqrt(1/a+1/b+1/c+1/d))
        else:
            return np.exp(np.log(OR)-1.96*np.sqrt(1/a+1/b+1/c+1/d))
    for cluster in count_diag_clusters:
        count_diag_notin=np.sum([count_diag_clusters[cl] for cl in clusters_ind if not(cl==cluster)])
        OR_diag[cluster]=(count_diag_clusters[cluster]/count_diag_notin)/((len(clusters[cluster])-count_diag_clusters[cluster])/np.sum([len(clusters[cl])-count_diag_clusters[cl] for cl in clusters_ind]))
        IC[cluster]={"up": IC_func("up",OR_diag[cluster],count_diag_clusters[cluster],(len(clusters[cluster])-count_diag_clusters[cluster]),count_diag_notin,np.sum([len(clusters[cl])-count_diag_clusters[cl] for cl in clusters_ind]))
                    ,"low": IC_func("low",OR_diag[cluster],count_diag_clusters[cluster],(len(clusters[cluster])-count_diag_clusters[cluster]),count_diag_notin,np.sum([len(clusters[cl])-count_diag_clusters[cl] for cl in clusters_ind]))}
    return OR_diag,IC

In [ ]:
OR_diag,IC=calculate_diag_OR(clusters_un,ind_groups,status)

In [ ]:
# Odds Ratio for clusters with more than 2 patients. If OR>1, a patient in the cluster is more likely to be diagnosed. The CI 
# is the confidence interval; if it does not span 1, then the OR is statistically significant
OR_diag,IC

In [ ]:
def phenotype_enrichment_analysis(patients_clustered,patient_phen,polarity_HPO):
    """Get the phenotypes shared by the most patients in the cluster according to polarity (positive or negative)
    Parameters: patients_clustered: list of patients in the cluster 
                patient_phen: dictionary of unique phenotypes associated with each patient; key is patient, value is dictionary
                with key "pos" or "neg" and value list of unique phenotypes with positive or negative association
                polarity_HPO: string, "pos" or "neg", polarity wanted for the phenotype enrichment analysis
    Returns: phen_ranked: list of best phenotypes ranked according to their representation in the cluster
             values: list of proportion of patients presenting the phenotype in the phen_ranked same position (ex: values[i]
             will have the represention of phenotype phen_ranked[i])
    """
    phen_count={}
    for patient in patients_clustered:
        for phen in patient_phen[patient][polarity_HPO]:
            if not(phen in phen_count):
                phen_count[phen]=1/len(patients_clustered)
            else:
                phen_count[phen]+=1/len(patients_clustered)
    phen_ranked=np.array([phen for phen in phen_count.keys()])
    values=np.array([phen_count[phen] for phen in phen_ranked])
    indrank=np.argsort(values)[::-1]
    phen_ranked=phen_ranked[indrank]
    values=values[indrank]
    return phen_ranked,values

In [ ]:
def get_HPO_count(patients_clustered,HPO_terms):
    """get the count of HPO terms for patients in the cluster, and the average
    Parameters: patients_clustered: dictionary with cluster number as key and list of patients in the cluster as value
                HPO_terms: dictionary with patient as key and count of HPO terms for the patient as value
    Returns: HPO_cluster: dictionary with cluster number as key and list of HPO numbers for each patient in the cluster as value
             avg_HPO_clusters: dictionary with cluster number as key and average number of HPO terms per patient as value
    """
    HPO_cluster = {i: [] for i in patients_clustered.keys()}
    for cluster in patients_clustered:
        for patient in patients_clustered[cluster]:
            HPO_cluster[cluster].append(HPO_terms[patient])
    avg_HPO_clusters = {cluster: np.average(HPO_cluster[cluster]) for cluster in patients_clustered.keys()}
    CI_HPO_clusters = {cluster: get_CI(HPO_cluster[cluster]) for cluster in patients_clustered.keys()}
    return HPO_cluster,avg_HPO_clusters,CI_HPO_clusters

In [ ]:
# get the ranked positively and negatively associated phenotyeps for patients in each cluster (phen_ranked_pos 
# and phen_ranked_neg) as well as count of HPO terms per patient (and average) for each cluster
# phen_ranked_pos (or _neg) is a dictionary with cluster number as key, and two arrays as value, one with the label
# of phenotypes ranked to their composition, another with the composition of said phenotype in the cluster
phen_ranked_pos,phen_ranked_neg={cluster: [] for cluster in ind_groups},{cluster: [] for cluster in ind_groups}
HPO_count,avg_HPO_clusters,CI_HPO_clusters=get_HPO_count(clusters_un,HPO_terms)
for cluster in ind_groups:
    phen_ranked_pos[cluster]=phenotype_enrichment_analysis(clusters_un[cluster],patient_phen,"pos")
    phen_ranked_neg[cluster]=phenotype_enrichment_analysis(clusters_un[cluster],patient_phen,"neg")
avg_HPO_clusters,CI_HPO_clusters

In [ ]:
def show_best_phenotypes_clusters(phen_ranked,nb):
    """Shows the nb best ranked phenotypes for each cluster that has ranked phenotypes
    Parameters: phen_ranked: dictionary with cluster number as key, two arrays as value, one with list of phenotypes 
                             ranked according to composition, second with composition of each phenotype
                nb: int, number of best phenotypes to show
    Returns: None
    Shows the nb best phenotypes for each cluster with their composition
    """
    for cluster in phen_ranked:
        print("Cluster ",cluster)
        print("Cluster len ",len(clusters_un[cluster]))
        n=(nb if len(phen_ranked[cluster][0])>10 else len(phen_ranked[cluster][0]))
        for i in range(n):
            print(phen_ranked[cluster][0][i],phen_ranked[cluster][1][i])

In [ ]:
show_best_phenotypes_clusters(phen_ranked_pos,10)

In [ ]:
show_best_phenotypes_clusters(phen_ranked_neg,10)

In [ ]:
def heatmap_phen(clusters_un,phen_ranked,figsize,vmin,vmax):
    """Displays heatmap of phenotype enrichment analysis for each cluster with analyzed composition
    Parameters: clusters_un: dictionary with cluster number as key and list of patients in the cluster as value
                phen_ranked: dictionary with cluster number as key, two arrays as value, one with list of phenotypes 
                             ranked according to composition, second with composition of each phenotype
                figsize: int, size of the figure displayed
                vmin: int, minimum value for the heatmap (here percentage)
                vmax: int, max value for the heatmap (here percentage)
    Returns: None
    Shows the heatmap of phenotype enrichment analysis for each cluster
    """
    cluster_list=["Cluster C"+str(cluster)+", size "+str(len(clusters_un[cluster])) for cluster in ind_groups]
    list_phen_max=[]
    for cluster in ind_groups:
        i,j=0,0
        while j<5:
            if not(phen_ranked[cluster][0][i]) in list_phen_max:
                list_phen_max.append(phen_ranked[cluster][0][i])
                j+=1
            i+=1
    heatmap_mat=[[] for i in range(len(list_phen_max))]
    for i,phen in enumerate(list_phen_max):
        for cluster in ind_groups:
            if phen in phen_ranked[cluster][0]:
                indphen=np.where(phen_ranked[cluster][0]==phen)[0][0]
                heatmap_mat[i].append(phen_ranked[cluster][1][indphen]*100)
            else:
                heatmap_mat[i].append(0)
    sns.set()
    fig,ax=plt.subplots(figsize=(figsize,figsize))
    sns.heatmap(heatmap_mat,cbar=True,cmap="YlGnBu",xticklabels=cluster_list,yticklabels=list_phen_max,ax=ax,vmin=vmin,vmax=vmax)
    plt.savefig("heatmap_all_clusters.png",bbox_inches="tight",dpi=350)
    plt.show()

In [ ]:
# heatmap for positive associations
heatmap_phen(clusters_un,phen_ranked_pos,12,0,75)

In [ ]:
# heatmap for negative associations
heatmap_phen(clusters_un,phen_ranked_neg,20,0,15)

In [ ]:
def metadata_collection(patients_clustered,metadata):
    """Get the metadata for each cluster 
    Parameters: patients_clustered: dictionary with cluster number as key and list of patients in the cluster as value
                metadata: dataframe with metadata
    Returns: metadata_clusters: dictionary with clusters as keys and dictionary as value, with key the metadata considered
                                and list of values for patients in the cluster as value
    """
    metadata_clusters={cl: {meta: [] for meta in list(metadata.columns)} for cl in patients_clustered.keys()}
    for cl in patients_clustered:
        for patient in patients_clustered[cl]:
            for meta in list(metadata.columns)[1:]:
                metadata_clusters[cl][meta].append(metadata.loc[patient][meta])
    return metadata_clusters

In [ ]:
# get the demographics for the patient in the cluster 
demographics_coll=metadata_collection(clusters_un,demographics)

In [ ]:
# show the average and 95% CI for age at UDN evaluation
for cluster in ind_groups:
    lst=np.array(demographics_coll[cluster]['\\00_Demographics\\Age at UDN Evaluation (in years)\\'])
    lst=lst[np.logical_not(np.isnan(lst))]
    print("Cluster C",cluster,"Average age at UDN eval : ",np.average(lst)," CI 95% : ",get_CI(lst))

In [ ]:
# show the average and 95% CI for age at symptom onset in years
for cluster in ind_groups:
    lst=np.array(demographics_coll[cluster]['\\00_Demographics\\Age at symptom onset in years\\'])
    lst=lst[np.logical_not(np.isnan(lst))]
    print("Cluster C",cluster,"Average age at onset : ",np.average(lst)," CI 95% : ",get_CI(lst))

In [ ]:
def get_distrib(attribute):
    """Get a distribution for an attribute
    Parameters: attribute: string, attribute we want the distribution of
    Returns: counter of the collection library (distribution)
    """
    counter={}
    for cluster in demographics_coll:
        dc=np.array(demographics_coll[cluster][attribute])
        if type(dc[0])==np.float64:
            dc=dc[np.logical_not(np.isnan(dc))]
        counter[cluster]=collec.Counter(dc)
    return counter

In [ ]:
gender_distrib=get_distrib('\\00_Demographics\\Gender\\')
gender_distrib

In [ ]:
age_eval_distrib=get_distrib('\\00_Demographics\\Age at UDN Evaluation (in years)\\')
for key in age_eval_distrib:
    print(age_eval_distrib[key])

In [ ]:
age_onset_distrib=get_distrib('\\00_Demographics\\Age at symptom onset in years\\')
age_onset_distrib

In [ ]:
race_distrib=get_distrib('\\00_Demographics\\Race\\')
race_distrib

### Statistics clusters

In [ ]:
ind_big_clusters=[0,2,3,4,5,6]

In [ ]:
# get the Kruskal Wallis for the distribution of HPO count between clusters
kruskal(HPO_count[0],HPO_count[2],HPO_count[3],HPO_count[4],HPO_count[5],HPO_count[6])

In [ ]:
def get_stats_value(value_considered):
    """get the Kruskal Wallis U index and p-value for a type of demographics
    Parameters: value_considered: string, type of demographics we want the KW U index and p-value
    Returns: None
    Prints the KW U index and p-value
    """
    dc={i: [] for i in ind_big_clusters}
    for i in ind_big_clusters:
        dc[i]=np.array(demographics_coll[i][value_considered])
        if type(dc[i][0])==np.float64:
            dc[i]=dc[i][np.logical_not(np.isnan(dc[i]))]
    print(kruskal(dc[0],dc[2],dc[3],dc[4],dc[5],dc[6]))

In [ ]:
# KW test for Age at UDN evaluation
get_stats_value('\\00_Demographics\\Age at UDN Evaluation (in years)\\')

In [ ]:
# KW test for Age at symptom onset
get_stats_value('\\00_Demographics\\Age at symptom onset in years\\')

In [ ]:
# KW test for Gender
get_stats_value('\\00_Demographics\\Gender\\')

In [ ]:
# KW test for Race
get_stats_value('\\00_Demographics\\Race\\')

## Link between clusters and diagnostics

In [ ]:
def get_list_diseases_from_XML(filename):
    """get the list of diseases in the Orphanet database by disease groups
    Parameters: string, file name for the Orphanet xml file
    Returns: list of diseases for the file name 
    """
    tree = ET.parse(filename)
    root = tree.getroot()
    dis=[]
    for s in root.findall(".//Disorder"):
        dis.append(s.find("Name").text.lower())
    return dis

In [ ]:
# get dictionary of all diseases in the Orphanet database, with disease group as key and list of diseases that belong
# to said disease group
all_diseases={}
for file in os.listdir("xml_orphanet/"):
    if file.endswith(".xml"):
        diseases=get_list_diseases_from_XML("xml_orphanet/"+str(file))
        all_diseases[file.split(".")[0]]=diseases
all_diseases

In [ ]:
# get the HPO mapping using the HPO database 
# mapping_HPO is a dictionary with HPO labels as key, dictionary as value, with keys id (value HPO number), key parent 
# (list of HPO numbers of n-1 parents), syn (possible label synonyms), xref (references to other classifications/ontologies)
mapping_HPO={}
with open("hpo.txt","r+") as hpo:
    lines = hpo.readlines()
    i=0
    for i in range(len(lines)):
        if lines[i].split(":")[0]=="id":
            hpoid=lines[i].split(" ")[1].split("\n")[0]
            name=""
            for namestr in lines[i+1].split(" ")[1:]:
                name+=namestr+" "
            name=name.split("\n")[0]
            mapping_HPO[name]={"id": hpoid, "xref": [], "parent": [], "syn":[]}
        if lines[i].split(" ")[0]=="xref:":
            mapping_HPO[name]["xref"].append(lines[i].split(" ")[1].split("\n")[0])
        if lines[i].split(" ")[0]=="is_a:":
            mapping_HPO[name]["parent"].append(lines[i].split(" ")[1])
        if lines[i].split(" ")[0]=="synonym:":
            namesyn=""
            for namestr in lines[i].split(" ")[1:]:
                namesyn+=namestr+" "
            namesyn=namesyn.split("\"")[1].split("'")[0]
            if len(namesyn.split("obsolete "))>1:
                namesyn=namesyn.split("obsolete ")[1]
            mapping_HPO[name]["syn"].append(namesyn)
mapping_HPO

In [ ]:
def get_diseases_cl(clusters,diagnostics):
    """get the list of diseases diagnosed for patients in the clusters
    Parameters: clusters: dictionary with cluster number as key and list of patients in the cluster as value
                diagnostics: dataframe with the associated diagnostics for each patient
    Returns: diseases_cl: dictionary with cluster number as key, dictionary as value, with patient ID as key, diagnosed
                          illness as value
    """
    diseases_cl={}
    for cluster in clusters:
        diseases_cl[cluster]={}
        for patient in clusters[cluster]:
            diag=diagnostics.loc[patient]["\\14_Disorders (in OMIM, from PhenoTips)\\"]
            if type(diag)==float:
                if not(np.isnan(diag)):
                    diseases_cl[cluster][patient]=diag
            else:
                if not(diag=="affected"):
                    diseases_cl[cluster][patient]=diag.lower()
    return diseases_cl

In [ ]:
diseases_cl=get_diseases_cl(clusters_un,diagnostics)

In [ ]:
def get_OMIM_orpha_map(OMIM_diseases):
    """Get the mapping between OMIM database and Orphanet database (for a single cluster)
    Parameters: OMIM_diseases: list of OMIM diseases labels we want the mapping for
    Returns: ORPHA_OMIM: dictionary with OMIM disease label as key and Orphanet disease label as value
    """
    ORPHA_OMIM={}
    for dis in OMIM_diseases:
        response = requests.get("https://api.omim.org/api/entry/search?search="+ dis +"&start=0&limit=2&apiKey=rLnJSxMySW68Y9vB8m5clA&include=externalLinks")
        if len(response.text.split("<orphanetDiseases>"))==1:
            print(dis)
        else:
            ORPHA_OMIM[dis]=response.text.split("<orphanetDiseases>")[1].split("</orphanetDiseases>")[0].split(";;")[-1]
    return ORPHA_OMIM

In [ ]:
def get_OMIM_ORPHA_link(diseases_cl):
    """get the entire mapping between the OMIM database and Orphanet database (for all clusters)
    Parameters: diseases_cl: dictionary with cluster number as key, dictionary as value, with patient ID as key and diagnosed
                             illness as value
    Returns: OMIM_ORPHA: dictionary with OMIM disease label as key and Orphanet diseases label as value
    """
    disOMIM=[]
    for cl in diseases_cl:
        disOMIM+=[diseases_cl[cl][patient].lower() for patient in diseases_cl[cl]]
    OMIM_ORPHA=get_OMIM_orpha_map(disOMIM)
    return OMIM_ORPHA

In [ ]:
OMIM_ORPHA=get_OMIM_ORPHA_link(diseases_cl)

In [ ]:
def get_disease_association_real(ind_groups,diseases_cl,all_diseases,OMIM_ORPHA):
    """get the disease associations for diagnosed diseases within the clusters
    Parameter: ind_groups: list of indices of clusters to consider
               diseases_cl: dictionary with cluster number as key, dictionary as value, with patient ID as key and diagnosed
                            illness as value
               all_diseases: dictionary with disease type as key, list of associated diseases as value
               OMIM_ORPHA: dictionary, mapping between OMIM diseases (key) and Orphanet diseases (value)
    Return: dg_associations_real: dictionary with cluster number as key, dictionary as value, with patient ID as key and
                                  disease type associated as value
    """
    dg_associations_real={}
    for cluster in ind_groups:
        dg_associations_real[cluster]={}
        for patient in diseases_cl[cluster]:
            dg_associations_real[cluster][patient]=[]
            for distype in all_diseases:
                if diseases_cl[cluster][patient] in OMIM_ORPHA:
                    if OMIM_ORPHA[diseases_cl[cluster][patient]].lower() in all_diseases[distype]:
                        dg_associations_real[cluster][patient].append(distype)
    return dg_associations_real
        

In [ ]:
dg_associations_real=get_disease_association_real(ind_groups,diseases_cl,all_diseases,OMIM_ORPHA)

In [ ]:
def get_count_dg(dg_associations,all_diseases):
    """Get the count of disease groups found associated with patients in the cluster
    Parameters: dg_associations: dictionary with cluster number as key, dictionary as value, with patient ID as key and
                                  disease type associated as value
                all_diseases: dictionary with disease type as key, list of associated diseases as value
    Returns: count_dg: dictionary with cluster number as key, dictionary as value, with disease group as key and count of disease
                        group 
    """
    count_dg={cluster: {} for cluster in patient_to_dg}
    for cluster in dg_associations:
        for patient in dg_associations[cluster]:
            for dg in dg_associations[cluster][patient]:
                if dg in count_dg[cluster]:
                    count_dg[cluster][dg]+=1
                else:
                    count_dg[cluster][dg]=1
    return count_dg

In [ ]:
count_dg_real=get_count_dg(dg_associations_real,all_diseases)
# print the disease groups for each cluster ranked, along with the count
for cluster in count_dg_real:
    print("Cluster ",cluster)
    list_dg=[]
    for dg in count_dg_real[cluster]:
        if not(dg=='rare_genetic_diseases'):
            list_dg.append([dg,count_dg_real[cluster][dg]])
    list_dg.sort(key = lambda x: x[1],reverse=True)
    print(list_dg)
    print("---------------------------------------------------------------------------------")

In [ ]:
def heatmap_real(count_dg_real,figsize,vmin,vmax):
    """Show the heatmap of cluster according to disease group composition
    Parameters: count_dg_real: dictionary with cluster number as key, dictionary as value, with disease group as key and count of disease
                               group
                figsize: int, size of the figure
                vmin: int, minimum value for the heatmap (here percentage)
                vmax: int, max value for the heatmap (here percentage)
    Returns: None
    Shows the heatmap associated to the composition in disease group for each cluster
    """
    type_disease={'rare_bone_diseases': 0, 'rare_skin_diseases': 1, 'rare_surgical_maxillofacial_diseases': 2, 
                  'rare_urogenital_diseases': 3, 'rare_neoplastic_diseases': 4, 'rare_surgical_thoracic_diseases': 5,
                  'rare_cardiac_malformations': 6, 'rare_teratologic_disorders': 7, 
                  'rare_systemic_and_rheumatological_diseases': 8, 'rare_sucking_swallowing_disorders': 9, 
                  'rare_hepatic_diseases': 10, 'rare_immunological_diseases': 11, 'rare_endocrine_diseases': 12, 
                  'chromosomal_anomalies': 13, 'rare_abdominal_surgical_diseases': 14, 'rare_infertility': 15, 
                  'rare_allergic_disease': 16, 'rare_haematological_diseases': 17, 'rare_genetic_diseases': 18, 
                  'rare_respiratory_diseases': 19, 'rare_neurological_diseases': 20, 'rare_renal_diseases': 21, 
                  'rare_infectious_diseases': 22, 'rare_rheumatologic_diseases_of_childhood': 23, 
                  'rare_inborn_errors_of_metabolism': 24, 'rare_developmental_anomalies_during_embryogenesis': 25, 
                  'rare_otorhinolaryngological_diseases': 26, 'rare_odontological_diseases': 27, 
                  'rare_gastroenterological_diseases': 28, 'rare_cardiac_diseases': 29, 
                  'rare_gynaecological_and_obstetric_diseases': 30, 'rare_intoxications': 31, 'rare_eye_diseases': 32}
    heatmap_mat=[[0 for i in range(len(count_dg_real))] for j in range(len(type_disease))]
    cluster_list=np.sort([cluster for cluster in count_dg_real])
    yl=["" for i in range(len(type_disease))]
    for dis in type_disease:
        listname=dis.split(".")[0].split("_")
        name=""
        for string in listname:
            name+=string+" "
        yl[type_disease[dis]]=name
    for cluster in count_dg_real:
        sumcluster=np.sum([count_dg_real[cluster][dg] for dg in count_dg_real[cluster]])
        for dg in count_dg_real[cluster]:
            index=cluster_list.tolist().index(cluster)
            heatmap_mat[type_disease[dg]][index]=count_dg_real[cluster][dg]/sumcluster*100
    cluster_list=["Cluster C"+str(cluster)+", size "+str(len(clusters_un[cluster])) for cluster in count_dg_real]
    sns.set()
    fig,ax=plt.subplots(figsize=(figsize,figsize))
    ax=sns.heatmap(heatmap_mat,cbar=True,xticklabels=cluster_list,yticklabels=yl,vmin=vmin,vmax=vmax)
    plt.xlabel("Clusters")
    plt.ylabel("Disease subgroup")
    plt.savefig("heatmap_disease_ass.png",bbox_inches="tight",dpi=500)
    plt.show()

In [ ]:
heatmap_real(count_dg_real,12,0,25)